<a href="https://colab.research.google.com/github/asusatijo/DS2002-Data-Project-1/blob/main/Aliza_and_Danella_DS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DS2002 Data Project

Group Names and Computing IDs

Danella Lei Romera (tzb5xh) and Aliza Susatijo (egg2qp)

In [14]:


import pandas as pd
import requests
import sqlite3
import json
import io


# Function to fetch data from an API URL
def fetch_data_from_api(api_url):
    response = requests.get(api_url)
    response.raise_for_status()  # Raise an error for HTTP issues
    return response.content


#Converts the data fetched from the api url into a dataframe
def convert_and_process_data(input_format, data):


    #checks whether the input format is csv or json
    if input_format == 'csv':
        return pd.read_csv(io.StringIO(data.decode('utf-8')))
    elif input_format == 'json':
        return pd.json_normalize(json.loads(data.decode('utf-8')))
    else:
        raise ValueError("Invalid Input: Input format must be csv or json")


# Modifies the dataframe based on what columns the user wants to keep and add. Then it saves the data to the output format specified
def modify_and_save_data(df, output_format, output_path=None, db_name=None, table_name=None, columns_to_keep=None, new_columns=None):


    #Prints a summary of ingestion data
    print(f"Ingestion Summary: Records = {len(df)}, Columns = {df.shape[1]}")
    print(df.columns)


    #Modifies what columns the dataframe keeps
    if columns_to_keep:
        df = df[columns_to_keep]  # Keep specified columns


    #Adds new columns
    if new_columns:
        for column_name, value in new_columns.items():
            df[column_name] = value


    # Saves data to the specified output format
    if output_format == 'csv':
        if output_path:
            df.to_csv(output_path, index=False)
            print(f"Data written to CSV at {output_path}")
        else:
            print(df.to_csv(index=False))


    elif output_format == 'json':
        if output_path:
            df.to_json(output_path, orient="records")
            print(f"Data written to JSON at {output_path}")
        else:
            df.to_json("project1.txt", orient="records")
            print("Data written to JSON at project1.txt")


    else:
        #Raises an error if no sql database name is provided
        if db_name:
            #connects to the database
            conn = sqlite3.connect(db_name)


            # Convert unsupported data types to strings before insertion
            for column in df.columns:
                if df[column].dtype not in [str, int, float]:  # Check if dtype is supported
                    df[column] = df[column].astype(str)  # Convert to string
            #saves dataframe to sql. Will use default table name if none is provided
            if table_name:
                df.to_sql(table_name, conn, if_exists='replace', index=False)
            else:
                df.to_sql("default_table", conn, if_exists='replace', index=False)


            conn.close()
            print(f"Data stored in SQL database: {db_name}, table: {table_name}")
        else:
          raise ValueError("Invalid Input: Make sure you enter in the database name")




    # Summary of post-processing
    print(f"Post-processing Summary: Records = {len(df)}, Columns = {df.shape[1]}")




#Asks the user and returns what input format, output format, columns of the data from the api they want. The user can also add new columns and data to the data frame
def interactive_etl_process(api_url):
    #Gets the input and output format from the user
    input_format = input("Enter input format (csv, json): ").lower()
    output_format = input("Enter output format (csv, json, sql): ").lower()


    #retrives the data from the api
    data = fetch_data_from_api(api_url)


    #converts the data into a dataframe
    df = convert_and_process_data(input_format, data)


    #prints out the columns of the DataFrame so the user can choose what columns to keep
    print(f"Columns in Table:  {df.columns.to_list()}")


    #Gets the columns the user wants from the data frame
    columns_to_keep = input("Enter columns to keep (comma-separated, or leave blank for all): ")
    if columns_to_keep:
        columns_to_keep = [col.strip() for col in columns_to_keep.split(',')]
    else:
        coloumns_to_keep = None


    #Gets the new columns and values the user wants to add to the data frame
    new_columns_input = input("Enter new columns to add in the format 'column_name:value' (comma-separated, or leave blank for none): ")
    new_columns = {}
    if new_columns_input:
        for item in new_columns_input.split(','):
            #makes sure input is in correct "column_name:value" format
            parts = item.split(':')
            if len(parts) != 2:
                raise ValueError("Input must be in the format 'column_name:value'.")
            column_name, value = parts
            new_columns[column_name.strip()] = value.strip()


    #Saves the data to the users specified database name and table name if the output format is sql
    #Saves the data to the output file path the user entered in if the output format is csv or json
    if output_format == 'sql':
        db_name = input("Enter the SQL database name (add .sql at the end): ")
        table_name = input("Enter name for table: ")
        modify_and_save_data(df, output_format, db_name=db_name, table_name=table_name, columns_to_keep=columns_to_keep, new_columns=new_columns)
    elif output_format == 'json' or output_format == 'csv':
        output_path = input("Enter the output file path (add .txt at the end): ")
        modify_and_save_data(df, output_format, output_path=output_path, columns_to_keep=columns_to_keep, new_columns=new_columns)
    else:
      raise ValueError("Invalid Input: Output format must be csv, json, or sql")




#processes data for the given api url
def main():
    api_URL = 'https://random-d.uk/api?format=json'
    interactive_etl_process(api_URL)


if __name__ == "__main__":
    main()

Enter input format (csv, json): json
Enter output format (csv, json, sql): sql
Columns in Table:  ['Base URL', 'Upload', 'v1.GET', 'v2.GET']
Enter columns to keep (comma-separated, or leave blank for all): 
Enter new columns to add in the format 'column_name:value' (comma-separated, or leave blank for none): 
Enter the SQL database name (add .sql at the end): file.sql
Enter name for table: duck
Ingestion Summary: Records = 1, Columns = 4
Index(['Base URL', 'Upload', 'v1.GET', 'v2.GET'], dtype='object')
Data stored in SQL database: file.sql, table: duck
Post-processing Summary: Records = 1, Columns = 4
